In [1]:
# eda_customer_sales.py
# ---------------------------------------
# Prepare Customer Sales dataset for modeling
# - Clean types (dates, numerics)
# - Handle missing values
# - Standardize numeric features (save scaler)
# - Save raw & standardized CSVs
# ---------------------------------------

import os
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

RAW_INPUT = "customer_intelligence_dataset.csv"

# 1) Load
df = pd.read_csv(RAW_INPUT)
print("Dataset shape:", df.shape)
print("First 5 rows:\n", df.head(5))
print("Columns:", df.columns.tolist())

# 2) Parse dates (safe)
for dc in ["sale_date", "last_purchase_date"]:
    if dc in df.columns:
        df[dc] = pd.to_datetime(df[dc], errors="coerce")

# 3) Select numeric features to scale
num_cols = ["price", "quantity", "total_value", "age", "tenure_months"]
existing_num_cols = [c for c in num_cols if c in df.columns]
missing_expected = [c for c in num_cols if c not in df.columns]
if missing_expected:
    print(f"⚠️ Missing expected numeric columns (will be skipped): {missing_expected}")

# 3a) Enforce numeric dtype where present
for c in existing_num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# 4) Handle missing values
#    - numeric -> median
#    - others -> most frequent (leave dates as-is; many models ignore them)
for c in existing_num_cols:
    med = df[c].median()
    df[c] = df[c].fillna(med)

# Most-frequent for categoricals (simple heuristic)
cat_cols = [c for c in df.columns if c not in existing_num_cols]
for c in cat_cols:
    if c in ["sale_date", "last_purchase_date"]:
        continue
    mode_val = df[c].mode(dropna=True)
    if not mode_val.empty:
        df[c] = df[c].fillna(mode_val.iloc[0])
    else:
        df[c] = df[c].fillna("Unknown")

# 5) Standardize numeric features (only those that exist)
scaler = StandardScaler()
if existing_num_cols:
    X_scaled = scaler.fit_transform(df[existing_num_cols])
    scaled_df_num = pd.DataFrame(X_scaled, columns=existing_num_cols, index=df.index)
else:
    scaled_df_num = pd.DataFrame(index=df.index)
    print("⚠️ No numeric columns available to scale.")

# 6) Build standardized dataframe: scaled numerics + untouched non-numerics
df_scaled = pd.DataFrame(index=df.index)
for c in df.columns:
    if c in existing_num_cols:
        df_scaled[c] = scaled_df_num[c]
    else:
        df_scaled[c] = df[c]

# 7) Save outputs (+ scaler for modeling)
os.makedirs("models", exist_ok=True)

df.to_csv("customer_sales_raw.csv", index=False)
df_scaled.to_csv("customer_sales_scaled.csv", index=False)

with open("models/kmeans_scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("\n✅ Saved:")
print(" - customer_sales_raw.csv (cleaned, original scales)")
print(" - customer_sales_scaled.csv (numeric features standardized)")
print(" - models/kmeans_scaler.pkl (use this in KMeans inference)")


Dataset shape: (8000, 18)
First 5 rows:
    sale_id customer_id product_id product_name     category  price  quantity  \
0  S000001   CUST01553       P004         Desk    Furniture  12000         2   
1  S000002   CUST00161       P010    Projector  Electronics  45000         2   
2  S000003   CUST00627       P010    Projector  Electronics  45000         4   
3  S000004   CUST01291       P009      Monitor  Electronics  12000         1   
4  S000005   CUST01456       P004         Desk    Furniture  12000         4   

    sale_date  total_value  age  gender region         segment  churn  \
0  2020-01-07        24000   40  Female   West  Small Business      1   
1  2023-11-10        90000   50  Female   West       Corporate      0   
2  2022-07-04       180000   33    Male  South       Corporate      0   
3  2023-10-18        12000   41    Male   West        Consumer      0   
4  2023-09-13        48000   60  Female   West        Consumer      0   

   tenure_months last_purchase_date    